In [ ]:
from io import BytesIO
import base64
from PIL import Image

from bokeh import plotting, palettes
from bokeh.models import HoverTool, ColumnDataSource, CategoricalColorMapper
import numpy as np
import pandas as pd

plotting.output_notebook()

In [ ]:
import matplotlib.colors as mplc
import matplotlib.pyplot as plt

In [ ]:
from captum.attr import *

In [ ]:
# need original image AND the attribution data to pull this off
def embeddableImage(image_cluster, attribution_cluster):
    # data should be [4, 3, 200, 1024] instance
    encoded_subimages = []
    for sub_image, sub_attribution in zip(image_cluster, attribution_cluster):
      # need to normalize each image to be between 0.0 to 1.0
      # can assume that everything is already a numpy array
      #norm = mplc.Normalize(vmin = np.amin(sub_image), vmax = np.amax(sub_image))
      #img_data = (norm(sub_image) * 255).astype(np.uint8).transpose(1, 2, 0)
      #image = Image.fromarray(img_data, mode='RGB')
      print(sub_image.shape)
      fig, _ = visualization.visualize_image_attr(sub_attribution.transpose(1,2,0), sub_image.transpose(1,2,0), method="blended_heat_map", sign="absolute_value", outlier_perc=0);
      buffer = BytesIO()
      #image.save(buffer, format='png')
      fig.savefig(buffer, format='png', bbox_inches='tight')
      encoded_subimages.append('data:image/png;base64,' + base64.b64encode(buffer.getvalue()).decode())
    return encoded_subimages

In [ ]:
import pandas as pd
import numpy as np

# x and attributions MUST have the same dimensions!
def umapPlot(embedding, x, y, attributions, yTrue=None, title=''):
    """ Plot the embedding of X and y with popovers using Bokeh """
    
    df = pd.DataFrame(embedding, columns=('x', 'y'))
    # for each image should be able to apply the embeddable image function
    # list of lists [rows x columns], x instances with 4 columns
    sub_images = np.array(list(map(embeddableImage, x, attributions)))
    for i in range(4):
      df['image'+str(i+1)] = sub_images[:,i]
    df['class'] = [str(d) for d in y]
    df['index'] = list(range(len(y)))
    if yTrue is not None:
        df['trueDigit'] = [str(d) for d in yTrue]

    datasource = ColumnDataSource(df)

    colorMapping = CategoricalColorMapper(factors=np.arange(10).astype(np.str), palette=palettes.Spectral10)

    plotFigure = plotting.figure(
        title=title,
        plot_width=600,
        plot_height=600,
        tools=('pan, wheel_zoom, reset')
    )

    if yTrue is None:
        tooltip = """
            <div>
                <div>
                    <img src='@image1' style='float: left; width:256px; height:50px; margin: 5px 5px 5px 5px'/>
                </div>
                <div>
                    <img src='@image2' style='float: left; width:256px; height:50px; margin: 5px 5px 5px 5px'/>
                </div>
                <div>
                    <img src='@image3' style='float: left; width:256px; height:50px; margin: 5px 5px 5px 5px'/>
                </div>
                <div>
                    <img src='@image4' style='float: left; width:256px; height:50px; margin: 5px 5px 5px 5px'/>
                </div>
                <div>
                    <span style='font-size: 16px; color: #224499'>Class:</span>
                    <span style='font-size: 18px'>@class</span>
                    <span style='font-size: 16px; color: #224499'>Index:</span>
                    <span style='font-size: 18px'>@index</span>
                </div>
            </div>
            """
    else:
        tooltip = """
            <div>
                <div>
                    <img src='@image' style='float: left; margin: 5px 5px 5px 5px'/>
                </div>
                <div>
                    <span style='font-size: 16px; color: #224499'>Digit:</span>
                    <span style='font-size: 18px'>@digit (true: @trueDigit)</span>
                </div>
            </div>
            """
    plotFigure.add_tools(HoverTool(tooltips=tooltip))

    plotFigure.circle(
        'x', 'y',
        source=datasource,
        color=dict(field='class', transform=colorMapping),
        line_alpha=0.6, fill_alpha=0.6, size=8
    )
    plotting.show(plotFigure)
    
    return plotFigure

In [ ]:
import torch
x = torch.load("x_subset.tensor")
y = torch.load("y_subset.tensor")

In [ ]:
combined_attributions = torch.load("integrated_gradients_combined_attributions.tensor")

In [ ]:
import umap
reducerFish = umap.UMAP(n_neighbors = 20,min_dist=0.5, verbose = True)
embeddingFish = reducerFish.fit_transform(torch.reshape(combined_attributions, (50, 2457600)))

In [ ]:
scaled_combined_attributions = combined_attributions + 0.3 # definitely improper but avoids 

In [ ]:
fig = umapPlot(embeddingFish, x.numpy(), y.squeeze().numpy(), scaled_combined_attributions.numpy(), title='UMAP projection of the Zebrafish dataset with Integrated Gradients Applied')

In [ ]:
from bokeh.io import show
show(fig)